In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import csv
from PIL import Image
import os
import librosa
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import cv2
import json
from keras import layers
from keras.layers import Input, Add, Dense, Activation, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, Sequential
from keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History
from keras.utils import layer_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.applications import EfficientNetB4
import pydot
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
f = open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}
real_labels

In [ ]:
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_df['class_name'] = train_df.label.map(real_labels)
train_df

In [ ]:
train,valid = train_test_split(train_df, test_size = 0.1, random_state = 42, stratify = train_df['class_name'])

In [ ]:
print(len(train))
print(len(valid))

In [ ]:
classes = len(train_df.label.unique())

In [ ]:
dataset_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

train_data = dataset_gen.flow_from_dataframe(train,
                             directory = '../input/cassava-leaf-disease-classification/train_images',
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = (128, 128),
                             color_mode= 'rgb',
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = 32)

valid_data = dataset_gen.flow_from_dataframe(valid,
                             directory = '../input/cassava-leaf-disease-classification/train_images',
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = (128, 128),
                             color_mode= 'rgb',
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = 32)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32,kernel_size =(3,3), padding = 'Same', activation = 'relu', input_shape = (128,128,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(256, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(256, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(512, (4,4), padding='Same', activation='relu'))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
SPE = train_data.n // train_data.batch_size
VS = valid_data.n // valid_data.batch_size

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True, verbose=1)

checkpoint = ModelCheckpoint("Trained_Best_Model.h5", save_best_only=True, monitor = 'val_loss', mode= 'min')

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 2, min_lr = 1e-6, mode = 'min', verbose = 1)

model.fit(train_data, validation_data = valid_data , epochs = 20, 
          batch_size = 32,
          callbacks=[early_stopping, checkpoint, reduce_lr] )

model.save('Trained_Final_Model.h5')

# **Predictions**

In [ ]:
img = cv2.imread('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = resize(img, (128, 128, 3))
img = img.reshape(1, 128, 128, 3)
pred_model = keras.models.load_model('/kaggle/working/Trained_Best_Model.h5')
y = model.predict(img)

In [ ]:
y

In [ ]:
y = np.argmax(y)
y

In [ ]:
test_dir = '../input/cassava-leaf-disease-classification/test_images/'
preds = [('image_id', 'label')]
for img in os.listdir(test_dir):
    path = test_dir + img
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize(image, (128,128,3))
    image = image.reshape(1, 128, 128, 3)
    y = model.predict(image)
    y = np.argmax(y)
    tup = (img, y)
    preds.append(tup)

In [ ]:
csvfile = open('submission.csv','w',newline='')
obj = csv.writer(csvfile)
for pred in preds:
    obj.writerow(pred)
csvfile.close()

In [ ]:
output1 = pd.read_csv('/kaggle/working/submission.csv')
output1